영화 추천 시스

In [1]:
# 구글 코랩 한글
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 21 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 1s (9,880 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/colab/dataanalysis

Mounted at /content/drive
/content/drive/MyDrive/colab/dataanalysis


In [3]:
!pip install folium
!pip install missingno

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import missingno as msno
import folium

### README

In [6]:
with open('./data/m1/README') as fp:
  readme =fp.read()

In [7]:
print(readme)

SUMMARY

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under the following conditions:

     * The user may not state or imply any endorsement from the
       University of Minnesota or the GroupLens Research Group.

     * The user must acknowledge the use of the data set in
       publications resulting from the use of the data set, and must
       send us an electronic or paper copy of those publications.

     * The user may not redistribute the data without separate
       permission.

     * The user may not use this information for any commercial or
       revenue-bearing purposes without first obtaining 

데이터 가져오기

In [8]:
# sep or delimiter : 필디를 구분하기 위해서 사용
# default : sep='.' delimiter = None
movies =pd.read_csv('./data/m1/movies.dat', delimiter='::', encoding='ISO-8859-1', engine='python', header=None)

In [9]:
print(movies.shape)

(3883, 3)


In [10]:
# MovieID::Title::Genres
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [11]:
movies.tail()

,0,1,2
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [12]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       3883 non-null   int64 
 1   1       3883 non-null   object
 2   2       3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [13]:
movies.set_index(0) # 머리 부분이 조금 다르다. 어떻게 다른거죠

,1,2
0,,
1,Toy Story (1995),Animation|Children's|Comedy
2,Jumanji (1995),Adventure|Children's|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama
5,Father of the Bride Part II (1995),Comedy
...,...,...
3948,Meet the Parents (2000),Comedy
3949,Requiem for a Dream (2000),Drama
3950,Tigerland (2000),Drama


In [14]:
users = pd.read_csv('./data/m1/users.dat', sep='::', engine='python', header=None)

In [15]:
#UserID::Gender::Age::Occupation::Zip-code
users.head()

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       6040 non-null   int64 
 1   1       6040 non-null   object
 2   2       6040 non-null   int64 
 3   3       6040 non-null   int64 
 4   4       6040 non-null   object
dtypes: int64(3), object(2)
memory usage: 236.1+ KB


In [17]:
ratings = pd.read_csv('./data/m1/ratings.dat',sep='::', engine='python',header=None)

In [18]:
#UserID::MOvieID::Rating::Timestamp
ratings.head()

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [19]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype
---  ------  --------------    -----
 0   0       1000209 non-null  int64
 1   1       1000209 non-null  int64
 2   2       1000209 non-null  int64
 3   3       1000209 non-null  int64
dtypes: int64(4)
memory usage: 30.5 MB


### 데이터 전처리(proeprocessing)   

In [20]:
print(dir(movies))

['T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_REVERSED', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmatmul__', '__rmod__', '__rmul

README을 읽고 column 이름을 변경한다.  

In [21]:
movies.columns # MovieID::Title::Genres

Int64Index([0, 1, 2], dtype='int64')

In [22]:
# 버전에 따라서는 컬러명이 숫자일때, 문자열로 써야하는 경우도 있다.
movies.rename(columns={0:'MovieID', 1:'Title', 2:'Genres'}, inplace=True)

In [23]:
movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [24]:
movies.tail()

,MovieID,Title,Genres
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [25]:
# UserID::MovieID::Rating::Timetamp

# column 이름을 바꿀 때는 columns 또는 axis=1을 사용한다.
# users.rename(columns={0:'UserID', 1:'Gender', 2:'Age', 3:'Occupation', 4:'Zip-code'}, inplace=True)
# users.rename(mapper={0:'UserID', 1:'Gender', 2:'Age', 3:'Occupation', 4:'Zip-code'}, axis=1, inplace=True)

#데이터를 읽어올때 column 이름을 지정할 수 있다.
users=pd.read_csv('./data/m1/users.dat', sep='::', engine='python', header =None, names=['UserID','Gender','Age','Occupation','Zip-code'])


In [26]:
users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [27]:
users.tail()

,UserID,Gender,Age,Occupation,Zip-code
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [28]:
# UserID::MovieID::Rating::Timestamp


#ratings.rename(columns={0:'USERID', 1:'MovieID', 2:'Rating', 3:'Timestamp'}, inplace=True)
#ratings.rename(mapper={0:'USERID', 1:'MovieID', 2:'Rating', 3:'Timestamp'}, axis=1, inplace=True)


ratings = pd.read_csv('data/m1/ratings.dat', 
                     sep='::', 
                    engine='python', 
                    header=None,
          names=['UserID', 'MovieID', 'Rating', 'Timestamp'])




In [29]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 데이터 구조 변경   

In [30]:
movies.columns

Index(['MovieID', 'Title', 'Genres'], dtype='object')

In [31]:
users.columns

Index(['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], dtype='object')

In [32]:
ratings.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

merge와 join 비교   

In [33]:
df = pd.DataFrame({'key':['K0','K1','K2','K3','K4','K5'],
                   'A':['A0','A1','A2','A3','A4','A5']})

df

,key,A
0,K0,A0
1,K1,A1
2,K2,A2
3,K3,A3
4,K4,A4
5,K5,A5


In [34]:
other = pd.DataFrame({'key':['K0','K1','K2'],
                   'B':['B0','B1','B2']})
other

,key,B
0,K0,B0
1,K1,B1
2,K2,B2


In [35]:
#outer join
df.join(other.set_index('key'), on='key')

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,NaN
4,K4,A4,NaN
5,K5,A5,NaN


In [36]:
other.join(df.set_index('key'), on='key')

,key,B,A
0,K0,B0,A0
1,K1,B1,A1
2,K2,B2,A2


In [37]:
#join()은 how속성의 값이 left이므로 left outer join을 실행한다.
#outer join
df.join(other.set_index('key'), on='key')

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2
3,K3,A3,NaN
4,K4,A4,NaN
5,K5,A5,NaN


In [38]:
# how = left
# Left Outer Join
other.join(df.set_index('key'),on='key')

,key,B,A
0,K0,B0,A0
1,K1,B1,A1
2,K2,B2,A2


In [39]:
# how 속성의 기본값이 inner이다.
# inner join
df.merge(other)

,key,A,B
0,K0,A0,B0
1,K1,A1,B1
2,K2,A2,B2


merge는 이름이 같은 column을 기준으로 데이터를 합친다.

In [40]:
data = ratings.merge(users).merge(movies)

In [41]:
data.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


In [42]:
#fancy indexing해서 필요한 컬럼만 가져온다.
recommendation_data = data[['UserID', 'MovieID','Rating']].copy()

In [43]:
recommendation_data.shape

(1000209, 3)

In [44]:
recommendation_data.head()

,UserID,MovieID,Rating
0,1,1193,5
1,2,1193,5
2,12,1193,4
3,15,1193,4
4,17,1193,5


join
- join은 index를 기준으로 데이터를 합쳐준다.  
- merge와 비슷하지만 중복되는 컬럼이 있으면 이름 구분자를 정해줘야 한다.(rsuffix, lsuffix)  

In [45]:
movies.columns

Index(['MovieID', 'Title', 'Genres'], dtype='object')

In [46]:
users.columns

Index(['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], dtype='object')

In [47]:
ratings.columns

Index(['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype='object')

In [48]:
users.head(1)

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067


In [49]:
# on ='UserID'는 ratings에서 UserID columns을 기준으로 삼는 옵션이다.
# other에서는 UserID를 인덱스로 사용한다.
# 즉, ratings와 users 모두 UserID를 기준으로 join한다.

# users.set_index를 인덱스로 설정한다.
ratings.join(other=users.set_index('UserID'), on ='UserID')

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,F,1,10,48067
1,1,661,3,978302109,F,1,10,48067
2,1,914,3,978301968,F,1,10,48067
3,1,3408,4,978300275,F,1,10,48067
4,1,2355,5,978824291,F,1,10,48067
...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,M,25,6,11106
1000205,6040,1094,5,956704887,M,25,6,11106
1000206,6040,562,5,956704746,M,25,6,11106
1000207,6040,1096,4,956715648,M,25,6,11106


In [50]:
# users에 UserID column이 있는 상태로 join하면 UserID라는 이름의 column이
# 양쪽 모두에 존재하기 때문에 Error발생
# rsuffix옵션으로 이름을 바꿔서 해결한다.
ratings.join(users, 'UserID', lsuffix='_')

,UserID_,MovieID,Rating,Timestamp,UserID,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,2.0,M,56.0,16.0,70072
1,1,661,3,978302109,2.0,M,56.0,16.0,70072
2,1,914,3,978301968,2.0,M,56.0,16.0,70072
3,1,3408,4,978300275,2.0,M,56.0,16.0,70072
4,1,2355,5,978824291,2.0,M,56.0,16.0,70072
...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,NaN,NaN,NaN,NaN,NaN
1000205,6040,1094,5,956704887,NaN,NaN,NaN,NaN,NaN
1000206,6040,562,5,956704746,NaN,NaN,NaN,NaN,NaN
1000207,6040,1096,4,956715648,NaN,NaN,NaN,NaN,NaN


pivot

In [54]:
#UserID를 index로, MovieID을 columns, Rating을 values로 설정한 평점표를 만든다.
recommendation_pivot = recommendation_data.pivot(index='UserID', columns='MovieID', values='Rating')

In [55]:
recommendation_pivot

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


missig data 처리
- drop이라는 이름이 붙으면 버리는 기능이다.  
- fill이라는 이름이 붙으면 채우는 기능이다.  

In [57]:
# NaN을 지정한 값으로 바꾼다.
# NaN을 0으로 바꾼다.
recommendation_pivot.fillna(0,inplace=True)

In [58]:
recommendation_pivot

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
data.head()

,UserID,MovieID,Rating,Timestamp,Gender,Age,Occupation,Zip-code,Title,Genres
0,1,1193,5,978300760,F,1,10,48067,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,M,56,16,70072,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,M,25,12,32793,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,M,25,7,22903,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,M,50,1,95350,One Flew Over the Cuckoo's Nest (1975),Drama


statck, unstack
- MultiIndex을 만든 다음, unstack으로 pivot과 똑같은 결과를 만들 수 있다. 

In [61]:
recommendation_data.head()

,UserID,MovieID,Rating
0,1,1193,5
1,2,1193,5
2,12,1193,4
3,15,1193,4
4,17,1193,5


In [62]:
# set_index에 이름을 여러개 넣으면 MultiIndex가 된다.
recommendation_data.set_index(['UserID', 'MovieID'])

,,Rating
UserID,MovieID,
1,1193,5
2,1193,5
12,1193,4
15,1193,4
17,1193,5
...,...,...
5949,2198,5
5675,2703,3
5780,2845,1


In [63]:
recommendation_data.head()

,UserID,MovieID,Rating
0,1,1193,5
1,2,1193,5
2,12,1193,4
3,15,1193,4
4,17,1193,5


In [64]:
# unstack은 index를 column으로 만들어준다.
# stack은 column을 index으로 만들어 준다.
data_unstack =recommendation_data.set_index(['UserID', 'MovieID']).unstack().copy() # 레벨이 userID가 0 movieID가 1인데 unstack하게 되면 movieID만 열로 바뀜

In [65]:
data_unstack.head()

Rating                                               ...            \
MovieID   1    2    3    4    5    6    7    8    9    10    ... 3943 3944   
UserID                                                       ...             
1          5.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
3          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
4          NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
5          NaN  NaN  NaN  NaN  NaN  2.0  NaN  NaN  NaN  NaN  ...  NaN  NaN   

                                                 
MovieID 3945 3946 3947 3948 3949 3950 3951 3952  
UserID                                           
1        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
5        NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 3706 columns]

In [66]:
data_unstack.fillna(0,inplace=True)

In [67]:
data_unstack.head()

Rating                                               ...            \
MovieID   1    2    3    4    5    6    7    8    9    10    ... 3943 3944   
UserID                                                       ...             
1          5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
2          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
3          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
4          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   
5          0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  0.0  0.0  ...  0.0  0.0   

                                                 
MovieID 3945 3946 3947 3948 3949 3950 3951 3952  
UserID                                           
1        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
5        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 3706 columns]

### 데이터 분석   

In [68]:
recommendation_pivot.head(2) # 데이터 전처리가 끝났다...

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# UserID가 2746인 사람이 본 영화 목록

mov_user_2746 = recommendation_pivot.loc[2746] # 2746에 해당하는 번호만 가져오기
# print(type(mov_user_2746))
# print(mov_user_2746) #Name: 2746, Length:3906, dtype : float64
mov_user_2746[recommendation_pivot.loc[2746]>0]


In [70]:
# UserID가 2746인 사람이 매긴 평점별 영화 개수
# recommendataion_pivot.loc[2746][recommendation_pivot.loc].values
recommendation_pivot.loc[2746][recommendation_pivot.loc[2746]>0].value_counts()

4.0    28
3.0    18
5.0    11
Name: 2746, dtype: int64

In [71]:
recommendation_pivot.loc[2746][recommendation_pivot.loc[2746]>0].unique()

array([5., 3., 4.])

In [74]:
# 함수를 정의해서 일반화 시키다.
def movie_seen(user_id):
  return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]

In [75]:
# 7번 유저가 본 영화
movie_seen(7)

MovieID
6       4.0
110     5.0
349     5.0
377     3.0
380     5.0
442     4.0
457     5.0
474     5.0
480     4.0
589     5.0
648     4.0
733     5.0
861     4.0
1196    5.0
1221    4.0
1270    4.0
1573    4.0
1580    4.0
1610    5.0
1722    4.0
1997    5.0
2028    5.0
2353    5.0
2571    5.0
2916    5.0
3107    3.0
3256    5.0
3418    3.0
3578    3.0
3753    4.0
3793    3.0
Name: 7, dtype: float64

corr
- corr은 column 기준으로 상관관계를 분석해준다.   

In [77]:
# 상관계수는 -1부터 1까지 표현. 1과 가까우면 연관관계, 0에 가까우면 관련이 없음.

# corr : correlationship(상관관계)
users[['UserID','Age']].corr()

,UserID,Age
UserID,1.000000,0.005713
Age,0.005713,1.000000


In [78]:
recommendation_pivot.head(1)

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [79]:
# UserID를 기준으로 상관관계를 구할 것이므로
# column이 UserID여야 한다.
# T: 전치행렬
small_test = recommendation_pivot.T.iloc[:500,:500]
small_test.head(1)

UserID,1,2,3,4,5,6,7,8,9,10,...,491,492,493,494,495,496,497,498,499,500
MovieID,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,5.0,5.0,...,2.0,0.0,0.0,0.0,3.0,5.0,0.0,0.0,3.0,0.0


In [80]:
small_test_corr = small_test.corr()
small_test_corr

# 상관관계가 높다 = 같은 영화를 많이 봤다는 것임.

UserID,1,2,3,4,5,6,7,8,9,10,...,491,492,493,494,495,496,497,498,499,500
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.016907,0.272729,0.323755,0.031705,0.348720,-0.011964,0.148394,0.238199,0.350593,...,0.136098,-0.005610,0.074962,-0.019674,0.418595,0.405654,-0.008031,0.252057,0.246475,-0.012313
2,-0.016907,1.000000,0.150573,0.182654,0.054504,0.048220,0.541906,0.208058,0.269339,0.224683,...,0.365915,-0.011860,0.261086,0.141897,0.269889,0.220967,0.170273,0.126130,0.157077,0.093592
3,0.272729,0.150573,1.000000,0.847461,-0.020928,-0.011348,0.148628,0.087742,0.114179,0.257507,...,-0.006605,-0.004845,0.017790,0.175622,0.624372,0.354638,-0.006936,0.174329,0.114293,-0.010634
4,0.323755,0.182654,0.847461,1.000000,-0.017066,-0.009254,0.178816,0.111312,0.139805,0.236773,...,-0.005386,-0.003951,0.029458,0.148203,0.738216,0.422349,-0.005656,0.210020,0.141176,-0.008672
5,0.031705,0.054504,-0.020928,-0.017066,1.000000,0.106377,0.043245,0.175314,0.277172,0.048702,...,0.013124,-0.016998,0.173495,-0.004569,-0.026715,0.030440,0.029029,0.288071,0.124601,0.318591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,0.405654,0.220967,0.354638,0.422349,0.030440,0.069678,0.253686,0.166248,0.408232,0.340741,...,0.229302,0.137432,0.069173,0.141922,0.525649,1.000000,0.262842,0.180319,0.117557,-0.022704
497,-0.008031,0.170273,-0.006936,-0.005656,0.029029,-0.013194,0.175702,0.078103,-0.017040,0.053932,...,0.266579,-0.005634,0.050662,0.084224,0.224140,0.262842,1.000000,-0.014999,-0.021007,-0.012365
498,0.252057,0.126130,0.174329,0.210020,0.288071,0.093908,-0.022345,0.269648,0.361255,0.125505,...,-0.014283,0.116613,0.241928,-0.036745,0.147060,0.180319,-0.014999,1.000000,0.143473,0.329550


UserID가 5인 사람과 상관관계가 높은 상위 10명을 뽑을 수 있다.

In [81]:
#sort_values()는 값의 크기순으로 데이터 순서를 정렬한다.
#ascending=False는 내림차순이다.
#default은 ascending=True로 오름차순이다.
small_test_corr[5].sort_values(ascending=False)[1:11]

UserID
193    0.462310
131    0.446335
281    0.404170
59     0.369056
225    0.360986
208    0.358564
30     0.350050
58     0.339585
23     0.339135
166    0.327965
Name: 5, dtype: float64

In [82]:
# 함수 정의를 통한 일반화
def nearest_user(user_id,n):
  return small_test_corr.loc[user_id].sort_values(ascending=False)[1:n+1]

5번 유저와 상관관계 높은 3명을 뽑아올 수 있다.

In [83]:
nearest_user(5,3)

UserID
193    0.462310
131    0.446335
281    0.404170
Name: 5, dtype: float64

상관관계가 가장 높은 193번과 5번 유저가 본 영화를 비교한다.

In [84]:
# 193번이 본 영화를 가져온다.
movie_seen(193)

MovieID
1       4.0
11      3.0
21      5.0
28      4.0
34      4.0
       ... 
3909    3.0
3911    4.0
3914    4.0
3927    3.0
3929    3.0
Name: 193, Length: 242, dtype: float64

In [85]:
# 5번이 본 영화를 가져온다.
movie_seen(5)

MovieID
6       2.0
16      3.0
24      1.0
29      5.0
32      4.0
       ... 
3728    2.0
3744    1.0
3786    3.0
3793    2.0
3799    3.0
Name: 5, Length: 198, dtype: float64

193번은 봤지만 5번 유저가 안 본 영화를 추천할 수 있다.   
5번은 봤지만 193번 유저가 안본 영화를 추천할 수 있다.    
(5번과 193번은 상관관계가 높았음으로 즉, 취향이 비슷한 사람이 본 영화를 추천)

In [87]:
# 193은 보고 5번은 보지 않은 영화 목록(MovieID)을 뽑아온다.

# 193번이 본 인덱스 값을 가지고 와서 5번이 본 인덱스 값을 뺀다.... 차집합 개념인듯
unseen = set(movie_seen(193).index) - set(movie_seen(5).index)
len(unseen)

193

In [88]:
unseen_movie = recommendation_data[recommendation_data.MovieID.isin(unseen)]
unseen_movie

,UserID,MovieID,Rating
5904,1,1197,3
5905,3,1197,5
5906,10,1197,5
5907,11,1197,5
5908,13,1197,4
...,...,...,...
980147,5405,3909,2
980148,5539,3909,1
980149,5874,3909,3
980150,5995,3909,3


In [93]:
# 193번 유저가 5점 준 영화
unseen_movie[(unseen_movie.Rating==5)&(unseen_movie.UserID==193)]

,UserID,MovieID,Rating
5970,193,1197,5
92658,193,1188,5
119918,193,1265,5
153586,193,21,5
207695,193,3671,5
210684,193,1136,5
342130,193,912,5
369213,193,1639,5
404071,193,1358,5
409578,193,1148,5


In [94]:
# 추천할  영화제목을 가져온다.
unseen_movie[(unseen_movie.Rating==5)&(unseen_movie.UserID==193)].merge(movies)

# movie에 해당하는 제목과 장르를 가지고 온다.

,UserID,MovieID,Rating,Title,Genres
0,193,1197,5,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
1,193,1188,5,Strictly Ballroom (1992),Comedy|Romance
2,193,1265,5,Groundhog Day (1993),Comedy|Romance
3,193,21,5,Get Shorty (1995),Action|Comedy|Drama
4,193,3671,5,Blazing Saddles (1974),Comedy|Western
5,193,1136,5,Monty Python and the Holy Grail (1974),Comedy
6,193,912,5,Casablanca (1942),Drama|Romance|War
7,193,1639,5,Chasing Amy (1997),Drama|Romance
8,193,1358,5,Sling Blade (1996),Drama|Thriller
9,193,1148,5,"Wrong Trousers, The (1993)",Animation|Comedy


In [96]:
# user_id(193), other_id(5)

# 함수 정의로 일반화
def unseen_movie(user_id, other_id):
  unseen = set(movie_seen(user_id).index)-set(movie_seen(other_id).index)
  unseen_movie = recommendation_data[recommendation_data.MovieID.isin(unseen)]
  return unseen_movie[(unseen_movie.Rating==5)&(unseen_movie.UserID==user_id)].merge(movies)

In [97]:
# 193번은 보고 5번은 보지 않은 영화 중에서 평점 5인 영화를 유저 5번에 추천
unseen_movie(193,5)

,UserID,MovieID,Rating,Title,Genres
0,193,1197,5,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
1,193,1188,5,Strictly Ballroom (1992),Comedy|Romance
2,193,1265,5,Groundhog Day (1993),Comedy|Romance
3,193,21,5,Get Shorty (1995),Action|Comedy|Drama
4,193,3671,5,Blazing Saddles (1974),Comedy|Western
5,193,1136,5,Monty Python and the Holy Grail (1974),Comedy
6,193,912,5,Casablanca (1942),Drama|Romance|War
7,193,1639,5,Chasing Amy (1997),Drama|Romance
8,193,1358,5,Sling Blade (1996),Drama|Thriller
9,193,1148,5,"Wrong Trousers, The (1993)",Animation|Comedy


### 실습

7번과 유사한 2명 뽑아서 둘 중 한명이라도 평점이 5점인 영화 추천

In [101]:
#(user_id) 7번이 본 영화(movieID)을 가져옴
def movie_seen(user_id):
  return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]

# 7번과 유사한 유저 2명 가져옴
def nearest_user(user_id,n): # n명을 가져온다.
  return small_test_corr.loc[user_id].sort_values(ascending=False)[1:n+1]

In [102]:
nearest_user(7,2)

UserID
288    0.676216
251    0.674230
Name: 7, dtype: float64

In [104]:
# 각각의 유저는 user0, user1로 해준다.
user0 = nearest_user(7,2).index[0]
user1 = nearest_user(7,2).index[1]

In [105]:
# user0(288)이 본 MovieID을 가져온다.
movie_seen(user0).index

Int64Index([  21,  110,  260,  349,  377,  380,  474,  480,  494,  733,  912,
            1036, 1101, 1127, 1196, 1197, 1198, 1200, 1210, 1222, 1240, 1242,
            1275, 1291, 1356, 1374, 1376, 1429, 1573, 1580, 1610, 1727, 1912,
            1918, 2000, 2001, 2058, 2135, 2167, 2273, 2278, 2353, 2406, 2571,
            2916, 2947, 2993, 3350, 3398, 3418, 3527, 3635, 3702, 3793, 3827,
            3911],
           dtype='int64', name='MovieID')

In [106]:
# user1(251)이 본 MovieID을 가져온다.
movie_seen(user1).index

Int64Index([   6,   11,   21,   32,  110,  210,  266,  316,  349,  368,  377,
             380,  383,  435,  457,  474,  480,  553,  589,  590,  594,  780,
            1047, 1079, 1081, 1220, 1233, 1259, 1266, 1270, 1307, 1320, 1370,
            1372, 1379, 1408, 1580, 1610, 1663, 1721, 1748, 1958, 1968, 2000,
            2012, 2028, 2046, 2058, 2065, 2094, 2174, 2474, 2716, 2795, 2804,
            2915, 2916, 2918, 3072, 3256, 3361, 3418, 3448, 3697, 3916, 3927,
            3928, 3930, 3936, 3937, 3946, 3948, 3952],
           dtype='int64', name='MovieID')

In [107]:
# user0(288)과 user1(251)이 본 영화중 5점인 영화 MovieID을 가져온다.

# user_id : 7
def recommend_movie(user_id, n):
  user0, user1 = nearest_user(user_id,n).index
  user_mv_list = recommendation_data[(recommendation_data.UserID.isin([user0, user1]))&(recommendation_data.Rating==5)]
  user7_mv_list = movie_seen(user_id)
  unseen_list=set(user_mv_list['MovieID'])-set(user7_mv_list.index)
  return movies[movies['MovieID'].isin(unseen_list)].reset_index(drop=True)

In [108]:
# 7번이 보지 않은 영화만 뽑았는지 검증(교집합의 결과는 없어야 한다.)
# 7번이 본 것을 가지고 오고, 7번과 유사한 2명이 본 것을 가지고 온다.
# 코드 출려 결과과 같은 값이 나오며 안된다.. 교집합
set(movie_seen(7).index)&set(recommend_movie(7,2)['MovieID'])

set()

In [109]:
recommend_movie(7,2)

,MovieID,Title,Genres
0,21,Get Shorty (1995),Action|Comedy|Drama
1,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1036,Die Hard (1988),Action|Thriller
3,1079,"Fish Called Wanda, A (1988)",Comedy
4,1101,Top Gun (1986),Action|Romance
5,1197,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
6,1198,Raiders of the Lost Ark (1981),Action|Adventure
7,1200,Aliens (1986),Action|Sci-Fi|Thriller|War
8,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
9,1240,"Terminator, The (1984)",Action|Sci-Fi|Thriller
